<a href="https://colab.research.google.com/github/pramsatriapalar/statistics_research_methodology_py.ipynb/blob/main/introduction_to_linear_and_polynomial_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to linear and polynomial regression

## Last updated: 24-March-2021

**Written by: [Pramudita Satria Palar ](https://sites.google.com/view/pramudita-satria-palar/home), Faculty of Mechanical and Aerospace Engineering, Bandung Institute of Technology**

#Initialize!

We will primarily use ```sklearn``` with some tools for (3D plotting ```mpl_toolkits```).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy import interpolate
from mpl_toolkits.mplot3d import Axes3D
import random
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

# Notes on notation

With regression, we are considering the following problem:

$y = f(\boldsymbol{x}) \approx \hat{f}(\boldsymbol{x})$,

where $\boldsymbol{x}$ is the input, $y$ is the output, and $f$ is a function that maps $\boldsymbol{x}$ into $y$. Finally, $\hat{f}$ is a regression model that approximates $f$

*   If you see $x_{ij}$, where $i$ and $j$ are integers: $i$ means the $i-$th observation for the $j-$th variable. I use this notation for multiple input variables. 
*   If you see $x_{i}$ (without any $j$), this means that this is the observation for one input variable. 
*   $y$ is used to denote the output.



# Simple linear regression (one variable)

Let's begin with a simple question, you have a variable $x$ and another variable $y$ that is a function of $x$ (i.e., $f(x)$). You want to know how $x$ relates to $y$ so you can gain more insight regarding this relationship or even make a prediction. The problem is, you don't know what $f(x)$ is really like. Worse, you cannot evaluate $f(x)$ at any locations as much as you like, but you can evaluate it at some locations, i.e., by collecting samples. 

For example, you have $y=f(x)=2x+4$, but you don't know this relationship. Instead, you collect $n$ samples (let's say $n=20$) and then evaluate it at some select points (notice that a Gaussian random noise $\mathcal{N}(0,1)$ is added to this function):


In [ ]:
#@title Creating the data set
np.random.seed(1) # To fix the random sample
nsamp = 20 # Number of samples
X = np.linspace(0,5,num=nsamp) # Create 100 samples between 0 and 5
noise = np.random.randn(nsamp)*1 # Noise term
y = 2*X+4 + noise # True function

# Plot figure
plt.figure(1,facecolor='white')
plt.scatter(X,y)
plt.show()


Based on these samples, it looks like that it's a linear function, right? But what is the best linear function that explain this relationship? Of course you can create as many lines as you like, but you will never know which line is the best until you have a guidance. To that end, we will employ the famous **linear regression** technique to capture this relationship. By linear regression you want to obtain a linear regression model $\hat{f}(x)$ (the 'hat' here means that it is an approximation model) that best approximates $f(x)$. The form of the linear model reads as

$\hat{f}(x) = \beta_{0}+\beta_{1}(x)$

where $\beta_{0}$ is the intercept and $\beta_{1}$ is the slope. By knowing $\beta_{0}$, you will know what value that $\hat{f}(x)$ will take when $x=0$. On the other hand, you will know how steep is the change in $\hat{f}(x)$ when you vary $x$ with small or large amount. You already have the samples, so let's build that model. For simplicity, let's call $\hat{y}_{i}$ as the linear regression prediction at sample $i$.

The question is: **What are the best values of $\beta_{0}$ and $\beta_{1}$?**. To answer this, we need to agree on something: **The best linear regression model is the model that minimizes the deviation between the linear regression line and the data**. Hence, if we define the error at sample $i$ as $\varepsilon_{i} = y_{i}-\hat{y}_{i}$. The best model is the model that minimizes the squared deviation between all samples and the linear regression line, that is

$\text{Minimize } \sum_{i=1}^{n}(e_{i})^{2} = \sum_{i=1}^{n}(y_{i}-\hat{y}_{i})^{2} = \sum_{i=1}^{n} (y_{i}-\beta_{0}-\beta_{1}x_{i})^{2}$

Notice that we replace $\hat{y}_{i}$ with $\beta_{0}+\beta_{1}x$ in the equation above. By using a simple derivative calculus we can obtain the expression to compute the optimal $\beta_{0}$ and $\beta_{1}$, that is

$\beta_{1} = \frac{\sum_{i=1}^{n}(x_{i}-\bar{x})(y_{i}-\bar{y})}{\sum_{i}^{n}(x_{i}-\bar{x})^{2}}$

$\beta_{0} = \bar{y}-\beta_{1}\bar{x}$

The calculation is very easy and I think you can code it by yourself. However, we will use ```sklearn``` to perform linear regression, because we will also use the same package for polynomial regression.

Remember that our data is generated from $y = 2x+4+\varepsilon$, where $\varepsilon$ is the noise term generated from a normal distribution $\mathcal{N}(0,1)$ (the true relationship is then $y = 2x+4$, the noise might come from measurement errors or other types of errors). We will use these 20 samples and then create a linear regression by using this data. We will use ```LinearRegression``` from ```sklearn``` to do just that. We will also create a linear regression called ```model``` by using ```LinearRegression```. You then need to fit the model by using our data set ```X``` and ```y```, by executing ```model.fit(X,y)```. After you fit the model, you can use ```model.predict()``` at any arbitrary location. Then, you can calculate the $R^{2}$ of the model by calling ```model.score(X,Y)``` (that will get you $R^{2}=0.88$, which is quite a good approximation) . Try that below:


In [ ]:
#@title Create a linear regression model with sklearn
# Reshape the data for sklearn
X = X.reshape(-1,1)
y = y.reshape(-1,1)

# Fit the linear regression model
model = LinearRegression() # Initialize linear regression model
model.fit(X, y) # Fit the model using the data

# Create validation samples
Xval = np.linspace(0,5,100).reshape(100,1)
yval = model.predict(Xval)
R2 = model.score(X,y)
# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(X,y) # Plot the training points
plt.plot(Xval,yval,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The model slope is {:.4f}'.format(model.coef_[0,0]))
print('The model intercept is {:.4f}'.format(model.intercept_[0]))
print('The R-squared of the model is {:.4f}'.format(R2))

You will obtain that the best linear regression model is $\hat{y} = 3.94 + 1.96 x$. This model tells you that if you have $x=0$, then $\hat{y}=3.94$. Furthermore, $\hat{y}$ will increase by 1.96 point if you increase $x$ by one point. In real-world context, this of course tells you about something about the problem.

Compare this with the true relationship, i.e., $y = 4 + 2x$. Not bad, eh? This linear regression model will be closer to the true relationship if the noise is smaller. How about if you try the following and observe what will happen:

*   Reduce the standard deviation of the noise. 
*   Increase the standard deviation of the noise.

The code below is how we can use a simple Python code to create a linear regression model without relying on ```sklearn```:


In [ ]:
#@title Calculate intercept and slope manually
# Calculate beta0 and beta 1 manually
beta1 = np.sum((X-np.mean(X))*(y-np.mean(y)))/ np.sum((X-np.mean(X))**2) # Intercept
beta0 = np.mean(y) - beta1*np.mean(X)
print('The model slope is {:.4f}'.format(beta1))
print('The model intercept is {:.4f}'.format(beta0))

Once you have the linear regression model, you can easily calculate the prediction at any arbitrary value of $x$! Well, we did that for our previous plot anyway, but let's try it again. If you use the linear regression model from ```sklearn```, we can use ```model.predict(x)``` (where ```x``` is an arbitrary value of ```x```). You can also input various values of $x$, to do that, please see how I create the linear regression plot above (i.e., that is, I inputted a 2-D Numpy array consisting of multiple sites of $x$)

In [ ]:
#@title Prediction at arbitrary locations

xpred = 4.1 # prediction site
ypred = model.predict(np.array([[xpred]])) # The input should be in the form of 2-D array
print('The prediction at x = {:.4f} is y = {:.4f}'.format(xpred, float(ypred)))

_____

**====ADVANCED BOX====**

Another alternative is to use the **ordinary least squares (OLS)**. This is more advanced, so if you are still not so familiar with the matrix algebra involved, that's fine, you can skip it for now. OLS is particularly useful in the following discussion when we talk about multiple linear regression and polynomial regression. So if you really want to understand both of them, don't skip this!

First, it is worth noting that OLS basically minimizes the squared of the differences between the data set (the output) and the linear model. The difference is that we solve the problem in a matrix form. Thus, in principle, it is still the same principle as we discussed earlier. The first step in OLS is to express everything in terms of matrix.

In the context of simple linear regression, we have two basis functions, i.e., $1$ and $x$. We can then input all of our samples into these two basis functions and we will obtain their values. We collect it into the so-called regression matrix $\boldsymbol{F}$ of size $n \times 2$:

$\boldsymbol{F} = \begin{bmatrix}1 & x_{1} \\
1 & x_{2}  \\
\vdots & \vdots \\
1 & x_{n}\end{bmatrix}$

we also have the so-called response vector $\boldsymbol{y}$ of size $n \times 1$:

$\boldsymbol{y} = \begin{bmatrix} y_{1} \\ y_{2} \\ \vdots \\ y_{n}\end{bmatrix}$

To obtain $\beta_{0}$ and $\beta_{1}$ which we also collect into a vector $\boldsymbol{\beta} = \{\beta_{0},\beta_{1}\}^{T}$, we need to solve the following overdetermined system:

$\boldsymbol{F}\boldsymbol{\beta}=\boldsymbol{Y}$

which can be done via the OLS technique, i.e.,

$\hat{\beta} = (\boldsymbol{F}^{T}\boldsymbol{F})^{-1} \boldsymbol{F}^{T} \boldsymbol{y}$

Although not detailed here, OLS uses the same principle of minimizing the squared deviation. We will then use OLS for multiple linear regression and also polynomial regression.

As for now, let's calculate the intercept and the slope from our previous data set by using OLS (remember that you need to run all previous cells first):

In [ ]:
#@title OLS for simple linear regression
F = np.concatenate((np.ones((nsamp,1)),X),axis=1)# Create the regression matrix
coef = np.linalg.inv(F.T@F) @ F.T @ y # Ordinary least squares
print('The model slope is {:.4f}'.format(coef[1,0]))
print('The model intercept is {:.4f}'.format(coef[0,0]))

____

# Multiple linear regression
You may have the following question: "Hey, using a linear regression for problems with one input variable seems cool. But how can I do it with multiple input variables?". You can create simple linear regression models for each of this variable. But be careful, such a simple approach fails in taking into account the full information from our data set. So how to deal with this?

Yes we have the answer for that: **Multiple linear regression** (let's call it MLR from now on). MLR is basically the extension of a simple linear regression for problems with multiple input variables. If you have $\hat{y} = \beta_{0} + \beta_{1}x$ for one input variable, we have the following expression for MLR with $m$ input variables:

$\hat{y} = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2} + \ldots + \beta_{m}x_{m}$

Similar to the simple linear regression, based on our data set, we need to find the coefficients $\boldsymbol{\beta} = \{\beta_{0},\beta_{1},\beta_{2},\ldots,\beta_{m}   \}^{T}$ that minimizes the deviation of the data set from the linear line (which is now more than a line, it is a plane - or hyperplane in higher dimension). Our data set now has $m$ input variables instead of just one, we collect it into an $n \times m$ matrix called $\boldsymbol{X}_{d}$:

$\boldsymbol{X}_{d} = \begin{bmatrix} x_{11} & x_{12} & \ldots & x_{1m}\\
x_{21} & x_{22} & \ldots & x_{2m} \\
\vdots & \vdots & \vdots & \vdots \\
 x_{n1} & x_{n2} & \ldots & x_{nm}\end{bmatrix}$

 where $x_{ij}$ indicates the $i$-th observation for the $j-$th variable.

We will use a function with two input variables to demonstrate the working principle of MLR. For two input variables, our MLR model is then: $\hat{y} = \beta_{0} + \beta_{1} x_{1} + \beta_{2} x_{2}$. The data set that we will use consists of two inputs (years of education and seniority) and one output (income). Therefore, we have the following relationship: $\text{income} = f(\text{education},\text{seniority}) = \hat{f}(\text{education},\text{seniority})$, where $\hat{f}$ is our MLR model. To put it more bluntly, our MLR model is:

$\hat{y} = \beta_{0} + \beta_{1} \times \text{education} + \beta_{2} \times \text{seniority}$

To create $\hat{y}$, we obtain data from 30 people, each with different years of education and seniority. Run the code below to create our data set (the first, second, and third column are years of education, seniority, and income):




In [ ]:
#@title CREATE THE INCOME DATA SET
income_data = np.array([[ 21.5862069,  113.10344828,  99.91717261],
 [ 18.27586207, 119.31034483,  92.57913486],
 [ 12.06896552, 100.68965517,  34.67872715],
 [ 17.03448276, 187.5862069 ,  78.70280624],
 [ 19.93103448,  20.        ,  68.00992165],
 [ 18.27586207,  26.20689655,  71.50448538],
 [ 19.93103448, 150.34482759,  87.97046699],
 [ 21.17241379,  82.06896552,  79.81102983],
 [ 20.34482759,  88.27586207, 90.00632711],
 [ 10.        , 113.10344828,  45.6555295 ],
 [ 13.72413793,  51.03448276,  31.91380794],
 [ 18.68965517, 144.13793103,  96.2829968 ],
 [ 11.65517241,  20.        ,  27.9825049 ],
 [ 16.62068966,  94.48275862,  66.60179242],
 [ 10.        , 187.5862069 ,  41.53199242],
 [ 20.34482759,  94.48275862,  89.00070082],
 [ 14.13793103,  20.        ,  28.81630076],
 [ 16.62068966,  44.82758621,  57.68169426],
 [ 16.62068966, 175.17241379,  70.10509604],
 [ 20.34482759, 187.5862069 ,  98.83401154],
 [ 18.27586207, 100.68965517,  74.7046992 ],
 [ 14.55172414, 137.93103448,  53.53210563],
 [ 17.44827586,  94.48275862,  72.07892367],
 [ 10.4137931 ,  32.4137931 ,  18.57066503],
 [ 21.5862069 ,  20.        ,  78.80578429],
 [ 11.24137931,  44.82758621,  21.38856131],
 [ 19.93103448, 168.96551724,  90.81403512],
 [ 11.65517241,  57.24137931,  22.63616262],
 [ 12.06896552,  32.4137931 ,  17.61359304],
 [ 17.03448276, 106.89655172,  74.6109602 ]])

Our data set is saved in the form of two-dimensional array. To be exact, our ```income_data``` data has 30 rows (meaning 30 observations) and 3 columns (one for years of education, one for seniority, one for income). Let's split them into the input matrix (```X_income```) and the output vector (```y_income```):

In [ ]:
#@title Splitting the data set
 
X_income = income_data[:,0:-1] # Take the YoE and seniority as the input
y_income = income_data[:,2] # Take the income as the output

If you want to know what is your data looks like:

In [ ]:
print(X_income)
print(y_income)

and then let's plot this data

In [ ]:
#@title Plot the multidimensional dataset
 
# Plot your data set 
fig = plt.figure(1,facecolor='white')
ax = fig.gca(projection='3d')
ax.scatter(X_income[:,0],X_income[:,1],y_income)
ax.set_xlabel('YoE')
ax.set_ylabel('Seniority')
ax.set_zlabel('Income')
plt.show()

Creating a MLR is surprisingly easy, the syntax is still the same with that of simple linear regression. The main difference is that your $X$ data now has $m$ variables instead of just 1. Now let's construct our MLR model and observe the intercept and the slopes:

In [ ]:
#@title Create a multiple linear regression model with sklearn
 
# Fit the linear regression model
modelMLR = LinearRegression() # Initialize linear regression model
modelMLR.fit(X_income, y_income) # Fit the model using the data
 
# Show the coefficients
print('The model slope for YoE is {:.4f}'.format(modelMLR.coef_[0]))
print('The model slope for seniority is {:.4f}'.format(modelMLR.coef_[1]))
print('The model intercept is {:.4f}'.format(modelMLR.intercept_))
print('The model R-squared is {:.4f}'.format(modelMLR.score(X_income,y_income)))

What does this intercept and slopes mean? The intercept ($\beta_{0}=-50.08$) simply means that when one does not have any education and seniority, he/she will receive an income of -50.08 (kind of does not make sense, I know). The slope for YoE, as indicated by $\beta_{1} = 5.8956$ simply means that one gets an extra 5.89 when he/she has one extra year of education. On the other hand, the slope for the seniority ($\beta_{2} = 0.1729$) shows that one gets an extra 0.17 income point when he/she has one more point of seniority. The $R^{2}=0.93$ is also quite high, indicating that our model is good enough. See? Those coefficients tell us something!

Let's create a plot for the MLR model that you just created

In [ ]:
#@title Plot the multiple linear regression model
# Create validation samples
Xplot = np.arange(10, 22, 0.25)
Yplot = np.arange(25, 175, 0.25)
Xplot, Yplot = np.meshgrid(Xplot, Yplot)
 
Zplot = np.zeros(Xplot.shape) # Initialize matrix for plotting
for ii in range(Xplot.shape[0]):
    for jj in range(Xplot.shape[1]):
        Zplot[ii,jj] = modelMLR.predict(np.array([[Xplot[ii,jj],Yplot[ii,jj]]]))
 
# Plot your data set 
fig = plt.figure(1,facecolor='white')
ax = fig.gca(projection='3d')
ax.plot_surface(Xplot,Yplot,Zplot)
ax.set_xlabel('YoE')
ax.set_ylabel('Seniority')
ax.set_zlabel('Income')
ax.scatter(X_income[:,0],X_income[:,1],y_income, label='Data points', marker='o')
ax.legend()
plt.show()

Doing a prediction is also super easy. Let's try it at $x = [20, 125]$ (i.e., YoE = 20 and seniority = 125), shall we?:

In [ ]:
#@title Prediction for the multiple linear regression model

xpred = np.array([[20,125]]) # prediction site, YoE = 20, seniority = 125
ypred = modelMLR.predict(xpred) # The input should be in the form of 2-D array
print('The prediction at x = {} is y = {:.4f}'.format(xpred, float(ypred)))

How about if you want to create an MLR model with more than two input variables? Easy peasy, you just need to change the number of columns of your $X$ matrix.

_____

**====ADVANCED BOX====**

Let's now go into the details of how an MLR model calculates the coefficients. It eventually boils down to **OLS** again. In terms of basis functions, our basis function is now $1$, $x_{1}$, $x_{2}$, $\ldots$, $x_{m}$, in which we need to calculate the coefficients of these basis functions. Therefore, our regression matrix $F$ is now
of size $n \times (1+m)$:

$\boldsymbol{F} = \begin{bmatrix}1 & x_{11} & x_{12} & \ldots & x_{1m}\\
1 & x_{21} & x_{22} & \ldots & x_{2m} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
1 & x_{n1} & x_{n2} & \ldots & x_{nm}\end{bmatrix}$

The response vector is still the same, $\boldsymbol{y}$ of size $n \times 1$:

We now have $\boldsymbol{\beta} = \{\beta_{0},\beta_{1},\beta_{2},\ldots,\beta_{m}\}^{T}$ that needs to be estimated by solving $\boldsymbol{F}\boldsymbol{\beta}=\boldsymbol{Y}$ via the OLS technique again.
_____


# Polynomial regression (one variable)
 
You might have the following question: Okay, linear model seems good if the true function is linear, but how if the function is nonlinear? One answer to that question (because we have so many answers!) is to use **polynomial regresion (PR)**. PR is of the form:
 
$\hat{f}(x)=\beta_{0} + \beta_{1}x + \beta_{2} x^{2} + \beta_{3} x^{3} + \ldots + \beta_{p}x^{p}$
 
where $p$ is the order of the polynomial. $p$ can be of any order and a good choice of $p$ is problem dependent. It is easy to see here that our linear regression model is a specific form of polynomial regression with $p=1$. We have the so-called quadratic equation by setting $p$ to 2 (i.e., we have $\hat{f}(x)=\beta_{0} + \beta_{1}x + \beta_{2} x^{2}$) and cubic by using $p=3$ (i.e., $\hat{f}(x)=\beta_{0} + \beta_{1}x + \beta_{2} x^{2} + \beta_{3} x^{3}$). PR is very useful if the true relationship is nonlinear, which is the reason why we put $x^{2}$ or other high order polynomials to the equation. Remember that the way we calculate the coefficients is also by using OLS.
 
Although useful, using a PR model might be tricky due to the following reasons:
 
*   To construct PR of order $p$, you need at least $1+n$ samples. In other words, you need more information to build good PR models!
*   There is a risk of **overfitting**  if you set a too high value of $p$! Why using a cubic model for a linear function after all? 
*   The PR model might also **underfit** the true function. For example, if your function is quadratic, using a linear model is not enough!
*   Determining the best order $p$ is not trivial. But luckily we have several methods that can help us (e.g., cross-validation)
 
But first, let's consider a quadratic PR:
 
$\hat{f}(x)=\beta_{0} + \beta_{1}x + \beta_{2} x^{2}$
 
and apply that in practice. Remember that our task is to calculate $\boldsymbol{\beta}$ based on our data set (in this context, $\boldsymbol{\beta}=\{\beta_{0},\beta_{1},\beta_{2}\}$). Let's say that our true function is $y = 0.2 x + 0.8 x ^{2} + \varepsilon$, where $\varepsilon$ is a noise term generated from a normal distribution $\mathcal{N}(0,2)$, evaluated within $[-3,5]$. We have 15 samples, let's try that:

In [ ]:
# Prepare the data set
np.random.seed(4) # Fix the random samples
nsamp = 15 # Number of samples
Xp = np.linspace(-3,5,nsamp).reshape(nsamp,1) # X data
sdnoise = 1.25 # Standard deviation of the noise
yp = 0.2*Xp + 0.8*(Xp**2) +  np.random.randn(nsamp,1)*sdnoise

# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xp,yp) # Plot the training points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend({'Data points'})
plt.show()

Clearly, if we force a linear model to this data set then we will obtain poor predictions. This is called **underfitting**:

In [ ]:
#@title Underfitting case
# Fit the linear regression model
model2 = LinearRegression() # Initialize linear regression model
model2.fit(Xp, yp) # Fit the model using the data

# Create validation samples
Xval2 = np.linspace(-3,5,200).reshape(200,1)
yval2 = model2.predict(Xval2)
R2 = model2.score(Xp,yp)
# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xp,yp) # Plot the training points
plt.plot(Xval2,yval2,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The model slope is {:.4f}'.format(model2.coef_[0,0]))
print('The model intercept is {:.4f}'.format(model2.intercept_[0]))
print('The R-squared of the model is {:.4f}'.format(R2))

Now, let's try to fit a quadratic model to this data set. In ```sklearn```, you need to add the so called ```polynomial_features``` to our linear regression model. The terminology is quite confusing but that's how ```sklearn``` deploys a PR model. We will also use the ```make_pipeline``` module from ```sklearn``` so that it is easier for us to construct a PR model. Let's create this PR model:

In [ ]:
#@title Prepare and fit the polynomial regression
p = 2 # Order of polynomial
polyreg = make_pipeline(PolynomialFeatures(degree=p), LinearRegression()) # Make an sklearn pipeline
polyreg.fit(Xp, yp) # Fit a polynomial regression model
Rsqrpolyreg = polyreg.score(Xp,yp)
modelPR = polyreg[1] # Extract the model
polymodel = polyreg[0] # Extract the polynomial setting

# Create validation samples (we use this for plotting purpose)
Xval = np.linspace(-3,5,200).reshape(200,1)
y_poly_pred = polyreg.predict(Xval) # Predict at Xval

# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xp,yp) # Plot the training points
plt.plot(Xval,y_poly_pred,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The R-squared of a polynomial regression model with p = {} is {:.4f}'.format(p, Rsqrpolyreg))
print('The PR model coefficients are {}'.format(modelPR.coef_[0,1:]))
print('The PR model intercept is {:.4f}'.format(float(modelPR.intercept_)))

With $p=2$, you will get the following polynomial model:

$\hat{f}(x) = -0.241 + 0.18 x + 0.807 x^{2}$

Compare this to the true function (without the noise term): $0.2 x + 0.8 x^{2}$. Surely not a really bad approximation, right? 

Careful, do no set $p$ too high. Otherwise, you will obtain predictions that **overfits!** The figure below is created using $p=13$ in which you can see that although the $R^{2}$ is high, the prediction is totally wrong!

Furthermore, you can change the value of $p$ (see ```p``` below) to see the effect of changing the polynomial order to the regression model:


In [ ]:
#@title Overfits model
p = 13 # Order of polynomial
polyreg = make_pipeline(PolynomialFeatures(degree=p), LinearRegression()) # Make an sklearn pipeline
polyreg.fit(Xp, yp) # Fit a polynomial regression model
Rsqrpolyreg = polyreg.score(Xp,yp)
modelPR = polyreg[1] # Extract the model
polymodel = polyreg[0] # Extract the polynomial setting

# Create validation samples (we use this for plotting purpose)
Xval = np.linspace(-3,5,200).reshape(200,1)
y_poly_pred = polyreg.predict(Xval) # Predict at Xval

# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xp,yp) # Plot the training points
plt.plot(Xval,y_poly_pred,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The R-squared of a polynomial regression model with p = {} is {:.4f}'.format(p, Rsqrpolyreg))
print('The PR model coefficients are {}'.format(modelPR.coef_[0,1:]))
print('The PR model intercept is {:.4f}'.format(float(modelPR.intercept_)))

# Cross-validation for model selection
Now you know that selection of $p$ is very important to ensure that your model is accurate. To that end, you need cross-validation which estimates the true test error by reusing your training data as validation data. We will directly use the feature from ```scikit-learn``` to calculate the cross-validation error. We will use the 5-fold cross-validation that calculates the cross-validated root-mean-squared-error (RMSE) to estimate the best polynomial order $p$. The best $p$, according to cross-validation, is the order that yields the lowest cross-validation error. Let's try that with our data by varying $p$ from $p=1$ to $p_{max}$.

In the code below, you only need to vary the $p_{max}$. The program will automatically calculate the $R^{2}$ and cross-validated RMSE for you. Remember that $R^{2}$ is calculated by using all your training set (without any folds) while RMSE is calculated by using five folds. Don't trust your $R^{2}$ when you want to select the best $p$, because even $p$ that overfits would yield $R^{2}\approx 1$!. Instead, use cross-validated RMSE for that purpose.

In [ ]:
pmax = 10 # you can try varying the pmax here.

scores = np.zeros((pmax,1))
Rsqrcv = np.zeros((pmax,1))
# Rsqrpolyreg = polyreg.score(Xp,yp)
for p in range(1,pmax+1):
    newmodel = LinearRegression()    
    poly = PolynomialFeatures(degree=p)
    Rsqrpolyregcv = polyreg.score(Xp,yp)
    X_current = poly.fit_transform(Xp)
    newmodel.fit(X_current,yp)
    scores[p-1,0] = np.mean(cross_val_score(model,X_current,yp,cv = 5,scoring ="neg_root_mean_squared_error"))
    Rsqrcv[p-1,0] = newmodel.score(X_current,yp)

bestindex = np.argmax(scores)
print("The best polynomial index is ",bestindex+1)
print(".. with cross-validated root mean squared error of ",-scores[bestindex])

Okay, so the program automatically performed everything for you. To get more sense about this, let's plot the polynomial order vs log-cross validation error (RMSE); note that we use log-scale for better visualization. Also, let's also plot the polynomial order vs $R^{2}$.

You will see that $R^{2}$ continuously increases while cross-validation error decreases and then increases again. The $p$ that yields the lowest cross-validation is then the best $p$ for your data set (the point with red colour)

In [ ]:
fig = plt.figure(1)
plt.plot(np.linspace(1,pmax,pmax,endpoint=True),np.log(-scores),marker='x')
plt.scatter(bestindex+1,np.log(-scores[bestindex]),color='red')
plt.ylabel('Log cross-validation error (RMSE)')
plt.xlabel('Polynomial order')
plt.show()

fig2 = plt.figure(2)
plt.plot(np.linspace(1,pmax,pmax,endpoint=True),Rsqrcv,marker='x')
plt.ylabel('R-squared')
plt.xlabel('Polynomial order')
plt.show()

_____

**====ADVANCED BOX====**

The regression matrix is slighty more complex for a one-dimensional polynomial regression. Now we have $p+1$ basis function, i.e., $1, x, x^{2},x^{3},\ldots,x^{p}$. We can then construct the $F$ matrix, in which its size is now $n \times p + 1$:


$\boldsymbol{F} = \begin{bmatrix}1 & x_{1} & x_{1}^{2} &  x_{1}^{3} & \ldots & x_{1}^{p}\\
1 & x_{2} & x_{2}^{2} &  x_{2}^{3} & \ldots & x_{2}^{p} \\
\vdots & \vdots & \vdots & \vdots & \ddots &\vdots \\
1 & x_{n}& x_{n}^{2} &   x_{n}^{3} & \ldots & x_{n}^{p}\end{bmatrix}$

For the system $\boldsymbol{F}\boldsymbol{\beta} = \boldsymbol{y}$ to be overdetermined and stable, you need at least sample size $n$ that is at least equals to $p+1$ (that is $n \geq p +1$). Otherwise, the system becomes highly unstable and your approximation becomes super bad! Want to try? Set $p$ in the problem above to $p=20$. You will get $R^{2} \approx 1$, but see how poor the prediction is.

_____


# Easy linear / polynomial regression
I made this box so that you can easily try any data set that you have. You can input your data in the form of matrix list or numpy array. In the example below, we have

$X = [0, 0.2, 0.3, 0.5, 0.7, 0.9, 1.2, 1.5]$

$y = [1, 2, 4, 5, 7, 6.5, 8, 9.6] $

that are saved in variable ```Xdata``` and ```ydata``` as a list, respectively. Just change the cell below if you want to use your own data set.

You can also input any function-generated data there too. For example, the following is for exponential function-generated data in $[0,3]$ with random noise $\mathcal{N}(0,0.2)$
```
Xdata = np.linspace(0,3, num = 100) 
ydata = np.exp(Xdata) + np.random.randn(Xdata.shape[0])*0.2
```



In [ ]:
# Don't change the name of the variables!
Xdata = [0, 0.2, 0.3, 0.5, 0.7, 0.9, 1.2, 1.5] # Your X data
ydata = [1, 2, 4, 5, 7, 6.5, 8, 9.6] # Your y data
p = 2 # Order polynomial

After that, just run the following cell so you can create any linear / polynomial regression! The program directly outputs the $R^{2}$ and the 5-fold cross validation too:

In [ ]:
#@title Easy linear / polynomial regression
Xdata = (np.array(Xdata)).reshape(-1,1)
ydata = (np.array(ydata)).reshape(-1,1)
polyreg2 = make_pipeline(PolynomialFeatures(degree=p), LinearRegression()) # Make an sklearn pipeline
polyreg2.fit(Xdata, ydata) # Fit a polynomial regression model
Rsqrpolyreg2 = polyreg2.score(Xdata,ydata)
modelPR2 = polyreg2[1] # Extract the model
polymodel2 = polyreg2[0] # Extract the polynomial setting
cverror = np.mean(cross_val_score(polyreg2,Xdata,ydata,cv = 5,scoring ="neg_root_mean_squared_error"))
 
# Create validation samples (we use this for plotting purpose)
lb = np.min(Xdata)
ub = np.max(Xdata)
Xval2 = np.linspace(lb,ub,200).reshape(200,1)
y_poly_pred2 = polyreg2.predict(Xval2) # Predict at Xval
 
# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xdata,ydata) # Plot the training points
plt.plot(Xval2,y_poly_pred2,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The R-squared of a polynomial regression model with p = {} is {:.4f}'.format(p, Rsqrpolyreg2))
print('The cross-validated RMSE error of a polynomial regression model with p = {} is {:.4f}'.format(p, -cverror))
print('The PR model coefficients are {}'.format(modelPR2.coef_[0,1:]))
print('The PR model intercept is {:.4f}'.format(float(modelPR2.intercept_)))

# Uploading from your drive
For this example, I use the 'income1.csv' data from this [reference](http://faculty.marshall.usc.edu/gareth-james/ISL/data.html). Execute the following cell to upload your data set (in the form of .csv):

In [ ]:
from google.colab import files

files.upload()

We need to make several arrangements to transform our csv data into a numpy array. Numpy array format is needed since ```sklearn``` works with Numpy array to process data. To that end, we can use ```np.genfromtxt``` [(see this for more details)](https://numpy.org/doc/stable/reference/generated/numpy.genfromtxt.html) so as to transform data from .csv to Numpy array. After that we need to split the data into $X$ and $y$:

In [ ]:
XYdata = np.genfromtxt('Income1.csv',skip_header = 1, delimiter = ',', usecols=[1,2])
Xdata = XYdata[:,0].reshape(-1,1)
ydata = XYdata[:,1].reshape(-1,1)

p = 2 # Polynomial order


and then we can do our easy linear / polynomial regression again.


In [ ]:
#@title Easy linear / polynomial regression
Xdata = (np.array(Xdata)).reshape(-1,1)
ydata = (np.array(ydata)).reshape(-1,1)
polyreg2 = make_pipeline(PolynomialFeatures(degree=p), LinearRegression()) # Make an sklearn pipeline
polyreg2.fit(Xdata, ydata) # Fit a polynomial regression model
Rsqrpolyreg2 = polyreg2.score(Xdata,ydata)
modelPR2 = polyreg2[1] # Extract the model
polymodel2 = polyreg2[0] # Extract the polynomial setting
 
# Create validation samples (we use this for plotting purpose)
lb = np.min(Xdata)
ub = np.max(Xdata)
Xval2 = np.linspace(lb,ub,200).reshape(200,1)
y_poly_pred2 = polyreg2.predict(Xval2) # Predict at Xval
 
# Plot the figure
plt.figure(1,facecolor='white')
plt.scatter(Xdata,ydata) # Plot the training points
plt.plot(Xval2,y_poly_pred2,'r-') # Plot the prediction points
plt.xlabel('x')
plt.ylabel('f(x)')
plt.show()
print('The R-squared of a polynomial regression model with p = {} is {:.4f}'.format(p, Rsqrpolyreg2))
print('The PR model coefficients are {}'.format(modelPR2.coef_[0,1:]))
print('The PR model intercept is {:.4f}'.format(float(modelPR2.intercept_)))